# ScoreCard Analysis Notebook

This notebook loads pickled pipeline state and performs comprehensive analysis including:
- Model validation metrics
- Prediction analysis
- GPT justification review
- Color change detection

**Prerequisites:** Run `scorecard_demo-2.ipynb` first to generate the state, then pickle it.

---
## 0. Save State (Run in Original Notebook First)

**Copy this cell to your source notebook and run it to save state:**

```python
from scorecard import save_state

# Save the full pipeline state
save_state(
    state=state,
    pipeline=pipeline,
    rag=rag,
    config=config,
    conn=conn,
    path="./pipeline_state.pkl",
    include_models=True,
    verbose=True,
)
```

---
## 1. Load Pickled State

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from scorecard import load_state

# Load the saved state
# Set reload_embeddings=True if you need to run GPT justifications
state, pipeline, rag, config, conn = load_state(
    path="./pipeline_state.pkl",
    reconnect=True,           # Re-establish ES/SQL/GPT connections
    reload_nlp=False,         # Skip spaCy (not needed for analysis)
    reload_embeddings=True,   # Needed for RAG/GPT operations
    verbose=True,
)

In [ ]:
# Quick verification of loaded state
print("\n" + "=" * 60)
print("LOADED STATE VERIFICATION")
print("=" * 60)

print(f"\nDataFrames:")
for name in ['enriched_df', 'predictions_df', 'complete_df']:
    df = getattr(state, name, None)
    if df is not None:
        print(f"  - {name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    else:
        print(f"  - {name}: None")

print(f"\nModel Horizons:")
for h, key in state.best_model_key_by_horizon.items():
    print(f"  - H{h}: {key[:60]}...")

print(f"\nPredictions by Horizon:")
for h, df in state.predictions_df_by_horizon.items():
    if df is not None:
        print(f"  - H{h}: {df.shape[0]:,} predictions")

---
## 2. Imports for Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import display, HTML, Markdown

from scorecard import (
    # Reporting
    enrich_for_reporting,
    generate_summary_tables,
    plot_prediction_dashboard,
    display_one_note,
    display_flagged_notes,
    generate_flagged_report,
    # Model validation
    compute_baseline_metrics,
    plot_baseline_comparison,
    compute_calibration_metrics,
    plot_calibration_curves,
    plot_precision_recall_curves,
    analyze_temporal_performance,
    plot_temporal_performance,
    analyze_errors,
    plot_error_analysis,
    extract_feature_importance,
    plot_feature_importance,
    generate_word_clouds,
    # RAG
    ScoreCardRag,
)

# Plot settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

print("All imports successful!")

---
## 3. Confusion Matrix Analysis

In [ ]:
print("=" * 70)
print("CONFUSION MATRIX ANALYSIS (Test Set)")
print("=" * 70)

color_labels = ["Green", "Yellow", "Red"]

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} - {state.best_model_key_by_horizon.get(h_int, 'Unknown')[:50]}...")
    print("=" * 70)
    
    y_test = model_dict.get("y_test")
    y_pred = model_dict.get("y_pred")
    
    if y_test is None or y_pred is None:
        print("  No test data available")
        continue
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Raw counts
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=color_labels, yticklabels=color_labels, ax=axes[0])
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].set_title(f'H{h_int} Confusion Matrix (Counts)')
    
    # Normalized
    sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=color_labels, yticklabels=color_labels, ax=axes[1])
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')
    axes[1].set_title(f'H{h_int} Confusion Matrix (Normalized)')
    
    plt.tight_layout()
    plt.show()
    
    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=color_labels))

---
## 4. Baseline Comparison

In [ ]:
print("=" * 70)
print("BASELINE COMPARISON")
print("=" * 70)
print("\nComparing our model against naive baselines:")
print("  - Most Frequent: Always predict the majority class")
print("  - Last Color: Predict the same color as previous note")
print("  - Stratified: Random prediction based on class distribution")

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} Baseline Comparison")
    print("=" * 70)
    
    y_test = model_dict.get("y_test")
    y_pred = model_dict.get("y_pred")
    y_proba = model_dict.get("y_proba")
    
    if y_test is None or y_pred is None:
        print("  No test data available")
        continue
    
    # Build temp dataframe for baseline comparison
    df_temp = pd.DataFrame({
        'true': y_test,
        'pred': y_pred,
    })
    
    if y_proba is not None:
        df_temp['prob_0'] = y_proba[:, 0]
        df_temp['prob_1'] = y_proba[:, 1] if y_proba.shape[1] > 1 else 0
        df_temp['prob_2'] = y_proba[:, 2] if y_proba.shape[1] > 2 else 0
    
    # Compute baseline metrics
    try:
        baseline_metrics = compute_baseline_metrics(
            df_temp, 
            true_col='true', 
            pred_col='pred',
            prob_cols=['prob_0', 'prob_1', 'prob_2'] if y_proba is not None else None
        )
        
        print(f"\nModel vs Baselines:")
        for name, metrics in baseline_metrics.items():
            acc = metrics.get('accuracy', 0)
            f1 = metrics.get('f1_macro', 0)
            print(f"  {name:20s}: Accuracy={acc:.3f}, F1-macro={f1:.3f}")
        
        # Plot comparison
        fig = plot_baseline_comparison(baseline_metrics)
        fig.suptitle(f"H{h_int} - Model vs Baselines", fontsize=14, fontweight='bold', y=1.02)
        plt.show()
        
    except Exception as e:
        print(f"  Error computing baselines: {e}")

---
## 5. Calibration Analysis

In [ ]:
print("=" * 70)
print("CALIBRATION ANALYSIS")
print("=" * 70)
print("\nWhen the model says '80% confident', is it right 80% of the time?")

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} Calibration")
    print("=" * 70)
    
    y_test = model_dict.get("y_test")
    y_proba = model_dict.get("y_proba")
    
    if y_test is None or y_proba is None:
        print("  No probability data available")
        continue
    
    try:
        # Build temp dataframe
        df_temp = pd.DataFrame({
            'true': y_test,
            'prob_0': y_proba[:, 0],
            'prob_1': y_proba[:, 1] if y_proba.shape[1] > 1 else 0,
            'prob_2': y_proba[:, 2] if y_proba.shape[1] > 2 else 0,
        })
        
        # Compute calibration metrics
        cal_metrics = compute_calibration_metrics(
            df_temp, 
            true_col='true',
            prob_cols=['prob_0', 'prob_1', 'prob_2']
        )
        
        print(f"\nCalibration Metrics per Class:")
        for cls, metrics in cal_metrics.items():
            ece = metrics.get('expected_calibration_error', 0)
            brier = metrics.get('brier_score', 0)
            print(f"  Class {cls}: ECE={ece:.4f}, Brier={brier:.4f}")
        
        # Plot calibration curves
        fig = plot_calibration_curves(df_temp, 'true', ['prob_0', 'prob_1', 'prob_2'])
        fig.suptitle(f"H{h_int} - Calibration Curves", fontsize=14, fontweight='bold', y=1.02)
        plt.show()
        
    except Exception as e:
        print(f"  Error: {e}")

---
## 6. Precision-Recall Curves

In [ ]:
print("=" * 70)
print("PRECISION-RECALL CURVES")
print("=" * 70)

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} Precision-Recall")
    print("=" * 70)
    
    y_test = model_dict.get("y_test")
    y_proba = model_dict.get("y_proba")
    
    if y_test is None or y_proba is None:
        print("  No probability data available")
        continue
    
    try:
        df_temp = pd.DataFrame({
            'true': y_test,
            'prob_0': y_proba[:, 0],
            'prob_1': y_proba[:, 1] if y_proba.shape[1] > 1 else 0,
            'prob_2': y_proba[:, 2] if y_proba.shape[1] > 2 else 0,
        })
        
        fig = plot_precision_recall_curves(df_temp, 'true', ['prob_0', 'prob_1', 'prob_2'])
        fig.suptitle(f"H{h_int} - Precision-Recall Curves", fontsize=14, fontweight='bold', y=1.02)
        plt.show()
        
    except Exception as e:
        print(f"  Error: {e}")

---
## 7. Error Analysis

In [ ]:
print("=" * 70)
print("ERROR ANALYSIS")
print("=" * 70)

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} Error Analysis")
    print("=" * 70)
    
    y_test = model_dict.get("y_test")
    y_pred = model_dict.get("y_pred")
    y_proba = model_dict.get("y_proba")
    
    if y_test is None or y_pred is None:
        print("  No test data available")
        continue
    
    try:
        # Calculate confidence
        if y_proba is not None:
            confidence = np.max(y_proba, axis=1)
        else:
            confidence = np.ones(len(y_pred))
        
        df_temp = pd.DataFrame({
            '_true': y_test,
            '_pred': y_pred,
            '_conf': confidence,
        })
        
        error_results = analyze_errors(df_temp, '_true', '_pred', '_conf')
        
        print(f"\n  Total Test Samples:     {error_results['total_samples']:,}")
        print(f"  Total Errors:           {error_results['total_errors']:,} ({error_results['error_rate']:.2%})")
        print(f"  High-Confidence Errors: {error_results['high_confidence_errors']:,} (conf >= 80%)")
        print(f"  Mean Confidence (Correct): {error_results['mean_conf_correct']:.3f}")
        print(f"  Mean Confidence (Errors):  {error_results['mean_conf_errors']:.3f}")
        
        print(f"\n  Error Breakdown:")
        for error_type, count in error_results['error_type_counts'].items():
            print(f"    {error_type:25s}: {count}")
        
        # Plot error analysis
        fig = plot_error_analysis(df_temp, '_true', '_pred', '_conf')
        fig.suptitle(f"H{h_int} - Error Analysis", fontsize=14, fontweight='bold', y=1.02)
        plt.show()
        
    except Exception as e:
        print(f"  Error: {e}")

---
## 8. Feature Importance (Top Predictive Words)

In [ ]:
print("=" * 70)
print("FEATURE IMPORTANCE - TOP PREDICTIVE WORDS")
print("=" * 70)

for h_int, model_dict in state.best_model_by_horizon.items():
    print(f"\n{'='*70}")
    print(f"H{h_int} Feature Importance")
    print("=" * 70)
    
    model = model_dict.get("model")
    vectorizer = model_dict.get("vectorizer") or state.vectorizer
    
    if model is None:
        print("  No model available")
        continue
    
    if vectorizer is None:
        print("  No vectorizer available")
        continue
    
    try:
        # Extract feature importance
        importance_df = extract_feature_importance(model, vectorizer, top_n=20)
        
        if importance_df is not None and len(importance_df) > 0:
            print(f"\n  Top 20 Features:")
            for _, row in importance_df.head(20).iterrows():
                print(f"    {row['feature']:30s} -> Class {int(row['class'])} (coef={row['coefficient']:.4f})")
            
            # Plot feature importance
            fig = plot_feature_importance(importance_df, top_n=15)
            fig.suptitle(f"H{h_int} - Top Predictive Words", fontsize=14, fontweight='bold', y=1.02)
            plt.show()
        else:
            print("  Could not extract feature importance")
            
    except Exception as e:
        print(f"  Error: {e}")

---
## 9. Color Change Prediction Analysis

In [ ]:
print("=" * 70)
print("COLOR CHANGE PREDICTION ANALYSIS")
print("=" * 70)

def analyze_color_changes(df, horizon_suffix=''):
    """Analyze predicted color changes."""
    results = {}
    
    # Get color columns
    last_color_col = 'last_color' if 'last_color' in df.columns else 'Overall'
    pred_color_col = f'pred_color{horizon_suffix}' if f'pred_color{horizon_suffix}' in df.columns else 'predicted_color'
    
    if last_color_col not in df.columns or pred_color_col not in df.columns:
        return None
    
    df_valid = df[[last_color_col, pred_color_col]].dropna()
    
    # Calculate changes
    df_valid['is_change'] = df_valid[last_color_col] != df_valid[pred_color_col]
    
    color_order = {'G': 0, 'Y': 1, 'R': 2}
    df_valid['last_ord'] = df_valid[last_color_col].map(color_order)
    df_valid['pred_ord'] = df_valid[pred_color_col].map(color_order)
    
    df_valid['is_downgrade'] = df_valid['pred_ord'] > df_valid['last_ord']
    df_valid['is_upgrade'] = df_valid['pred_ord'] < df_valid['last_ord']
    
    results['total'] = len(df_valid)
    results['changes'] = df_valid['is_change'].sum()
    results['downgrades'] = df_valid['is_downgrade'].sum()
    results['upgrades'] = df_valid['is_upgrade'].sum()
    results['no_change'] = results['total'] - results['changes']
    
    return results

# Analyze for each horizon
for h_int, pred_df in state.predictions_df_by_horizon.items():
    if pred_df is None:
        continue
    
    print(f"\n{'='*70}")
    print(f"H{h_int} - Color Change Analysis")
    print("=" * 70)
    
    results = analyze_color_changes(pred_df)
    if results:
        print(f"\n  Total Predictions:   {results['total']:,}")
        print(f"  Predicted Changes:   {results['changes']:,} ({results['changes']/results['total']:.1%})")
        print(f"    - Downgrades:      {results['downgrades']:,}")
        print(f"    - Upgrades:        {results['upgrades']:,}")
        print(f"  No Change:           {results['no_change']:,} ({results['no_change']/results['total']:.1%})")

---
## 10. Prediction Dashboard

In [ ]:
print("=" * 70)
print("PREDICTION DASHBOARD")
print("=" * 70)

# Prepare data for reporting
if state.predictions_df is not None:
    report_df = state.predictions_df.copy()
    
    # Enrich for reporting
    try:
        enriched_report = enrich_for_reporting(report_df)
        
        # Generate summary tables
        summaries = generate_summary_tables(enriched_report)
        
        print("\nSummary Tables:")
        for name, summary_df in summaries.items():
            print(f"\n{name}:")
            display(summary_df)
        
        # Generate dashboard
        fig = plot_prediction_dashboard(enriched_report)
        plt.show()
        
    except Exception as e:
        print(f"Error generating dashboard: {e}")
else:
    print("No predictions_df available")

---
## 11. GPT Justification Analysis

This section demonstrates GPT-powered explanations for model predictions.

In [ ]:
print("=" * 70)
print("GPT JUSTIFICATION SETUP")
print("=" * 70)

# Create RAG object if not loaded
if rag is None and conn is not None:
    print("\nCreating RAG object...")
    rag = ScoreCardRag(config=config, state=state, conn=conn)
    print("RAG object created!")
elif rag is not None:
    print("\nRAG object already loaded!")
else:
    print("\nWARNING: Cannot create RAG - no connection available")
    print("Re-run load_state with reconnect=True and reload_embeddings=True")

In [ ]:
# Helper function to display GPT justification nicely
def display_gpt_justification(sid_key, rag, state, show_context=True):
    """
    Generate and display a GPT justification for a specific sid_key.
    """
    print("=" * 70)
    print(f"GPT JUSTIFICATION FOR: {sid_key}")
    print("=" * 70)
    
    # Get the row from complete_df
    df = state.complete_df
    if df is None:
        print("ERROR: complete_df not available")
        return
    
    row = df[df['sid_key'] == sid_key]
    if row.empty:
        print(f"ERROR: sid_key '{sid_key}' not found")
        return
    
    row = row.iloc[0]
    
    # Display basic info
    print(f"\n--- Basic Information ---")
    print(f"SID: {row.get('SID', 'N/A')}")
    print(f"Vendor: {row.get('Supplier_Name', row.get('LM_Vendor_ID', 'N/A'))}")
    print(f"Program: {row.get('Program_Name', 'N/A')}")
    print(f"Note Date: {row.get('Note_Year', 'N/A')}-{row.get('Note_Month', 'N/A')}")
    
    # Display prediction
    print(f"\n--- Model Prediction ---")
    print(f"Predicted Color: {row.get('predicted_color', 'N/A')}")
    print(f"Confidence: Green={row.get('prob_green', 0):.1%}, Yellow={row.get('prob_yellow', 0):.1%}, Red={row.get('prob_red', 0):.1%}")
    print(f"Actual Color: {row.get('Overall', 'N/A')}")
    print(f"Color History: {row.get('color_set', 'N/A')}")
    
    # Display the note text
    if show_context:
        print(f"\n--- Scorecard Note ---")
        note_text = row.get('Scorecard_Note', row.get('pre_scrub_text', 'N/A'))
        print(note_text[:1000] + "..." if len(str(note_text)) > 1000 else note_text)
    
    # Generate GPT justification
    print(f"\n--- GPT Justification ---")
    print("Generating justification (this may take a moment)...\n")
    
    try:
        # Get augmented history
        history = rag.retrieve_augmented_history(sid_key)
        
        # Generate justification
        rag.generate_justifications(sid_key, printer=True)
        
    except Exception as e:
        print(f"ERROR generating justification: {e}")
    
    print("\n" + "=" * 70)

In [ ]:
# Get sample sid_keys for GPT analysis
print("=" * 70)
print("SAMPLE PREDICTIONS FOR GPT ANALYSIS")
print("=" * 70)

if state.complete_df is not None:
    df = state.complete_df
    
    # Find interesting cases
    print("\n1. High-confidence predictions (>80%):")
    if 'prob_green' in df.columns:
        high_conf = df[
            (df['prob_green'] > 0.8) | 
            (df['prob_yellow'] > 0.8) | 
            (df['prob_red'] > 0.8)
        ].head(5)
        if not high_conf.empty:
            display(high_conf[['sid_key', 'SID', 'predicted_color', 'prob_green', 'prob_yellow', 'prob_red']].head())
            sample_high_conf = high_conf['sid_key'].iloc[0]
            print(f"\nSample high-confidence sid_key: {sample_high_conf}")
    
    print("\n2. Predicted color changes (downgrades):")
    if 'predicted_color' in df.columns and 'Overall' in df.columns:
        downgrades = df[
            ((df['Overall'] == 'G') & (df['predicted_color'].isin(['Y', 'R']))) |
            ((df['Overall'] == 'Y') & (df['predicted_color'] == 'R'))
        ].head(5)
        if not downgrades.empty:
            display(downgrades[['sid_key', 'SID', 'Overall', 'predicted_color', 'prob_green', 'prob_yellow', 'prob_red']].head())
            sample_downgrade = downgrades['sid_key'].iloc[0]
            print(f"\nSample downgrade sid_key: {sample_downgrade}")
    
    print("\n3. Recent predictions (for review):")
    recent = df.sort_values(['Note_Year', 'Note_Month'], ascending=False).head(5)
    display(recent[['sid_key', 'SID', 'Note_Year', 'Note_Month', 'predicted_color', 'Overall']].head())
else:
    print("No complete_df available")

In [ ]:
# Generate GPT justification for a specific sid_key
# Replace with an actual sid_key from your data

# Example: Use a sample from the data
if state.complete_df is not None and rag is not None:
    # Get a sample sid_key (you can change this to any specific one)
    sample_sid_key = state.complete_df['sid_key'].iloc[0]
    
    print(f"Generating GPT justification for: {sample_sid_key}")
    print("(Change sample_sid_key to analyze a different note)\n")
    
    display_gpt_justification(sample_sid_key, rag, state, show_context=True)
else:
    print("Cannot generate justification - missing data or RAG object")

---
## 12. Batch GPT Justifications

In [ ]:
# Generate justifications for multiple predictions
# WARNING: This can be slow and consume API credits!

def batch_generate_justifications(sid_keys, rag, state, max_count=5):
    """
    Generate GPT justifications for multiple sid_keys.
    """
    print("=" * 70)
    print(f"BATCH GPT JUSTIFICATIONS (max {max_count})")
    print("=" * 70)
    
    results = []
    for i, sid_key in enumerate(sid_keys[:max_count]):
        print(f"\n[{i+1}/{min(len(sid_keys), max_count)}] Processing: {sid_key}")
        try:
            rag.generate_justifications(sid_key, printer=False)
            results.append({'sid_key': sid_key, 'status': 'success'})
            print(f"  -> Success!")
        except Exception as e:
            results.append({'sid_key': sid_key, 'status': 'error', 'error': str(e)})
            print(f"  -> Error: {e}")
    
    print(f"\n" + "=" * 70)
    success_count = sum(1 for r in results if r['status'] == 'success')
    print(f"Completed: {success_count}/{len(results)} successful")
    
    return results

# Example: Generate for top 5 high-confidence predictions
# Uncomment to run:
# if state.complete_df is not None and rag is not None:
#     sample_keys = state.complete_df['sid_key'].head(5).tolist()
#     batch_results = batch_generate_justifications(sample_keys, rag, state, max_count=5)

---
## 13. Retrieve Stored Justifications from Elasticsearch

In [ ]:
def get_stored_justification(sid_key, conn, config):
    """
    Retrieve a previously generated justification from Elasticsearch.
    """
    es = conn.es_client
    try:
        result = es.get(index=config.rag_index, id=sid_key)
        source = result['_source']
        return {
            'sid_key': sid_key,
            'justification': source.get('justification', source.get('gpt_justification', '')),
            'note': source.get('Scorecard_Note', ''),
            'created_at': source.get('created_at', ''),
        }
    except Exception as e:
        return {'sid_key': sid_key, 'error': str(e)}

def display_stored_justification(sid_key, conn, config):
    """
    Display a stored justification nicely.
    """
    result = get_stored_justification(sid_key, conn, config)
    
    print("=" * 70)
    print(f"STORED JUSTIFICATION: {sid_key}")
    print("=" * 70)
    
    if 'error' in result:
        print(f"Error: {result['error']}")
        return
    
    print(f"\nCreated: {result.get('created_at', 'Unknown')}")
    print(f"\n--- Note ---")
    print(result.get('note', 'N/A')[:500])
    print(f"\n--- Justification ---")
    print(result.get('justification', 'No justification stored'))
    print("=" * 70)

# Example usage:
# if conn is not None:
#     display_stored_justification('000001.2024.06.000001', conn, config)

---
## 14. Interactive Analysis

Use the cells below for ad-hoc analysis.

In [ ]:
# Quick data exploration
print("Available DataFrames in state:")
for attr in ['enriched_df', 'predictions_df', 'complete_df', 'sid_df', 'details_df']:
    df = getattr(state, attr, None)
    if df is not None:
        print(f"  - state.{attr}: {df.shape}")
        print(f"    Columns: {list(df.columns)[:5]}...")

In [ ]:
# Analyze a specific SID
def analyze_sid(sid, state):
    """Show all predictions for a specific SID."""
    df = state.complete_df
    if df is None:
        print("No data available")
        return
    
    sid_data = df[df['SID'] == sid].sort_values(['Note_Year', 'Note_Month'])
    
    if sid_data.empty:
        print(f"No data for SID {sid}")
        return
    
    print(f"\nSID {sid} - {len(sid_data)} notes")
    print(f"Vendor: {sid_data.iloc[0].get('Supplier_Name', 'N/A')}")
    print(f"Program: {sid_data.iloc[0].get('Program_Name', 'N/A')}")
    
    cols = ['sid_key', 'Note_Year', 'Note_Month', 'Overall', 'predicted_color', 'prob_green', 'prob_yellow', 'prob_red']
    available_cols = [c for c in cols if c in sid_data.columns]
    display(sid_data[available_cols])

# Example:
# analyze_sid(1, state)

In [ ]:
# Custom GPT analysis cell - modify sid_key as needed
# sid_key = "000001.2024.01.000001"  # Replace with actual sid_key
# display_gpt_justification(sid_key, rag, state)

---
## 15. Summary

In [ ]:
print("=" * 70)
print("ANALYSIS SESSION SUMMARY")
print("=" * 70)

print(f"\nData Loaded:")
if state.enriched_df is not None:
    print(f"  - Enriched notes: {len(state.enriched_df):,}")
if state.predictions_df is not None:
    print(f"  - Predictions: {len(state.predictions_df):,}")
if state.complete_df is not None:
    print(f"  - Complete dataset: {len(state.complete_df):,}")

print(f"\nModels:")
for h, key in state.best_model_key_by_horizon.items():
    print(f"  - H{h}: {key[:50]}...")

print(f"\nConnections:")
print(f"  - Elasticsearch: {'Connected' if conn and conn.es_client else 'Not connected'}")
print(f"  - GPT Client: {'Available' if conn and conn.gpt_client else 'Not available'}")
print(f"  - RAG: {'Ready' if rag else 'Not initialized'}")

print(f"\n" + "=" * 70)
print("Analysis complete!")
print("=" * 70)